# Load Path

In [1]:
import sys
import os
from PIL import Image
import shutil

GOOGLE_DEFAULT_PATH="/content/drive/"
GOOGLE_PROJECT_PATH="My Drive/"
GOOGLE_LESSION_PATH="CNN程式設計概念"
GOOGLE_TMP_NAME="My Drive/course_python_tmp/"

GOOGLE_MY_PATH = GOOGLE_DEFAULT_PATH+GOOGLE_PROJECT_PATH+GOOGLE_LESSION_PATH
MY_PATH_TMP = None

# its win32, maybe there is win64 too?
is_windows = sys.platform.startswith('win')

if is_windows:
    if os.path.exists('./tmp_3'):
      shutil.rmtree('./tmp_3')
    pass
    os.makedirs('./tmp_3')
    MY_PATH_TMP = "./tmp_3/"
else:
    from google.colab import drive
    from PIL import Image

    # 1. mount google drive
    drive.mount(GOOGLE_DEFAULT_PATH, force_remount=True)
    os.chdir(GOOGLE_MY_PATH) #change to this path
    Curr_Path=os.getcwd()
    print(Curr_Path)

    MY_PATH_TMP = GOOGLE_DEFAULT_PATH+GOOGLE_TMP_NAME

    # 2. create tmp
    if not os.path.exists(MY_PATH_TMP):
        os.makedirs(MY_PATH_TMP)
    pass

pass

Curr_Path=os.getcwd()
print(Curr_Path)

C:\Users\marti\OneDrive\桌面\CNN程式設計概念\Brain_Tumor_detection\federated learning pytorch


套件導入

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import random
import os

###########################
from tqdm import tqdm

# pytorch 函式庫
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import torch.optim as optim

# 借用tensorflow的影像處理函式庫
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical

# 用來調整X/Y的函式庫
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

# 用來儲存樣本的函式庫
import pickle # serialization

# 載入黑白AI模型

In [3]:
def Image_to_Array(pic, width, height):
    image = load_img(pic, target_size=(width, height),color_mode='grayscale')
    image_array = img_to_array(image)
    image_preprocess = preprocess_input(image_array) # normalize the value of each element in array
    return  image_preprocess
pass

def load_pictures(picdir, width_size, height_size):

    # init variables to store sample
    x_data = []
    y_labels = []

    for dirPath, dirNames, fileNames in os.walk(picdir):
        for f in fileNames:
            # get y (ans)
            ans = int(dirPath[len(picdir):])

            # get x (picture)
            PicDir = os.path.join(dirPath, f)
            NArray_2D  = Image_to_Array(PicDir,width_size,height_size)

            # store y, x
            y_labels.append(ans)
            x_data.append(NArray_2D)
        pass
    pass
    return y_labels, x_data
pass

def encode_data(i_y, i_x, i_dim):
    # convert the data and labels to NumPy arrays
    x_data = np.array(i_x, dtype="float32")
    y_labels = np.array(i_y)
    #################################################
    # perform one-hot encoding on the labels
    # from 3 to [0,0,0,1,0,0,0,0,0,0]
    #################################################
    y_labels3 = to_categorical(y_labels,i_dim)

    return y_labels3, x_data
pass

# 把眾多照片存成一個檔案
def save_sample(i_y, i_x, i_filename):
    f = open(i_filename, 'wb')
    pickle.dump([i_y,i_x], f)
    f.close()
pass

# 把那一個檔案中讀出來眾多照片
def load_sample(i_filename):
    f = open(i_filename, 'rb')
    r = pickle.load(f)
    y_labels = r[0]
    x_data = r[1]
    return y_labels, x_data
pass

# 在眾多照片中隨機抽樣
def get_sample(i_y, i_x, i_sample_size):
    ##############################
    # get training data randomly

    # init x batch and y batch
    x_batch=[]
    y_batch=[]

    sample = random.sample(range(len(i_y)), i_sample_size)

    x_batch = [i_x[i] for i in sample]
    y_batch = [i_y[i] for i in sample]

    return np.array(y_batch), np.array(x_batch)
pass

# 抽樣完後，把照片轉成tensor格式
def Image_to_Tensor(pic, width_size, height_size, depth_size=1):
    image = None
    if depth_size == 1: # 黑白
        image = load_img(pic, target_size=(width_size, height_size),color_mode='grayscale')
    else : # 彩色
        image = load_img(pic, target_size=(width_size,height_size))
    pass
    image_array = img_to_array(image)
    image_preprocess = preprocess_input(image_array) # normalize the value of each element in array
    # x.reshape(batch, width, height, bw=1) color=4
    x = image_preprocess.reshape(-1, width_size,height_size,depth_size)
    x = np.transpose(x, (0, 3, 1, 2))
    x = torch.from_numpy(x).float()
    return x
pass


class Graph_pytorch(nn.Module):
    def __init__(self):
        super(Graph_pytorch, self).__init__() # customize model
        self.var = OrderedDict()

        #self.op = OrderedDict()

        self.op_sampling = OrderedDict()
        self.op_linear = OrderedDict()

        self.lr = 0.001 # learning rate
    pass
pass

def Graph_pytorch_set_input_size(self, i_input_size):
    self.NETWORK_INPUT_FRAME_WIDTH = i_input_size[0]
    self.NETWORK_INPUT_FRAME_HEIGHT = i_input_size[1]
    self.NETWORK_INPUT_FRAME_DEPTH = i_input_size[2]
pass
Graph_pytorch.set_input_size=Graph_pytorch_set_input_size

def Graph_pytorch_set_output_size(self, i_output_dim):
    self.NETWORK_OUTPUT_DIM=i_output_dim
pass
Graph_pytorch.set_output_size=Graph_pytorch_set_output_size


def Graph_pytorch_init_op_sampling(self):

    self.op_sampling['dropout'] = nn.Dropout2d(p=0.3)
    # conv 1
    _layer_1_input = self.NETWORK_INPUT_FRAME_DEPTH
    _layer_1_output = 32
    self.op_sampling['conv_1'] =  nn.Conv2d(in_channels=_layer_1_input,
                      out_channels=_layer_1_output,
                      kernel_size=5,
                      stride=2,
                      padding=0,
                      bias=True)

    # pool 1
    filter_w=2
    filter_h=2
    stride_x=2
    stride_y=2
    self.op_sampling['pool_1']  = nn.MaxPool2d((filter_w, filter_h),
                       stride=(stride_x, stride_y),
                       padding=0)

    # conv 2
    _layer_3_input = _layer_1_output
    _layer_3_output = 64
    self.op_sampling['conv_2'] =  nn.Conv2d(in_channels=_layer_3_input,
                      out_channels=_layer_3_output,
                      kernel_size=5,
                      stride=2,
                      padding=0)

    # pool 2
    self.op_sampling['pool_2'] = nn.MaxPool2d((filter_w, filter_h),
                      stride=(stride_x, stride_y),
                      padding=0)
pass
Graph_pytorch.init_op_sampling = Graph_pytorch_init_op_sampling


def Graph_pytorch_get_linear_relation_layer_input_size(self):

    i_x = np.zeros((1,  self.NETWORK_INPUT_FRAME_DEPTH,
            self.NETWORK_INPUT_FRAME_WIDTH,
            self.NETWORK_INPUT_FRAME_HEIGHT),dtype=float)
    x = self.to_placeholder(i_x)
    # 產出經過samping的樣本
    for index, layer in self.op_sampling.items():
        x = layer(x)
    pass

    # 找出經過sampling的樣本的大小

    _batch, _width, _height, _depth = list(x.size())

    # 這個經過sampling的樣本，需要拉平成一維矩陣，才能丟進去線性關係式
    # 線性關係式: linear relation / full connected layer / fc layer
    # 下面的步驟是找出一維矩陣的大小
    self.Linear_Relation_INPUT_SIZE = _width * _height * _depth
pass
Graph_pytorch.get_linear_relation_layer_input_size = Graph_pytorch_get_linear_relation_layer_input_size

def Graph_pytorch_init_op_linear(self):
    # 建立多項式 (fc2) Y=AX+b，其中有300個x，Y是1x10的矩陣
    _input_shape = self.Linear_Relation_INPUT_SIZE # 須根據sampling後的大小來手動設定
    _ouput_shape = self.NETWORK_OUTPUT_DIM
    _hidden_layer_shape_1 = 256

    self.op_linear['fc1'] = nn.Linear(_input_shape, _hidden_layer_shape_1)

    self.op_linear['fc2'] = nn.Linear(_hidden_layer_shape_1, _ouput_shape)
    self.op_linear['softmax'] = nn.Softmax(dim=1)
pass
Graph_pytorch.init_op_linear = Graph_pytorch_init_op_linear


def Graph_pytorch_init_optimizer(self):
    # 指定我們的優化器為 Adam，需要告訴優化器
    # variable是誰(也就是告訴優化器A和b是哪個變數)
    _parameters = []

    for key, value in self.op_sampling.items():
        _parameters.append({'params':value.parameters()})
    pass

    for key, value in self.op_linear.items():
        _parameters.append({'params':value.parameters()})
    pass

    self.optimizer = optim.Adam(_parameters,
                  lr=self.lr, weight_decay=0)
pass
Graph_pytorch.init_optimizer = Graph_pytorch_init_optimizer


def Graph_pytorch_forward(self, i_x):

    # (batch, bw color, width, height)
    x = torch.reshape(i_x,(-1,self.NETWORK_INPUT_FRAME_DEPTH
            ,self.NETWORK_INPUT_FRAME_WIDTH
            ,self.NETWORK_INPUT_FRAME_HEIGHT))

    for index, value in self.op_sampling.items():
        x = value(x)
    pass

    _linear_input_shape = self.Linear_Relation_INPUT_SIZE # 須根據sampling後的大小來手動設定
    x = x.view(-1, _linear_input_shape)

    for index, value in self.op_linear.items():
        x = value(x)
    pass
    return x
pass
Graph_pytorch.forward = Graph_pytorch_forward

def Graph_pytorch_to_placeholder(self, i_x):
    return torch.tensor(i_x, requires_grad=False,dtype=torch.float)
pass
Graph_pytorch.to_placeholder = Graph_pytorch_to_placeholder


def Graph_pytorch_backward(self, i_loss):
    # 反向傳播
    i_loss.backward()
    # 更新模型參數
    self.optimizer.step()
    # 清空梯度
    self.optimizer.zero_grad()
pass
Graph_pytorch.backward = Graph_pytorch_backward


def Graph_pytorch_cal_loss(self, Y_sample, Y_prediction):
    error = Y_sample - Y_prediction
    loss = (error ** 2).mean()
    return loss
pass
Graph_pytorch.cal_loss = Graph_pytorch_cal_loss

將圖片設置為.p，以利之後讀取(若已有要訓練的圖片集檔案可跳過此步)

若已有要訓練的圖片集檔案可跳至此步

In [4]:
y_labels,x_data = load_sample('data/Brain_Tumor_classify.p')
y_labels_glioma_tumor,x_data_glioma_tumor = load_sample('data/glioma_tumor.p')
y_labels_meningioma_tumor,x_data_meningioma_tumor = load_sample('data/meningioma_tumor.p')
y_labels_pituitary_tumor,x_data_pituitary_tumor = load_sample('data/pituitary_tumor.p')

環境設置

In [5]:
############################
# 0. environment setting
# size of picture
width_size=60
height_size=80
depth_size=1 # bw=1, color=3
output_dim=4 # bw tumor img class=4 (0~3), 分類

############################
# 2. data cleaning

# 2.1. data cleaning for x and y
y_c_labels, x_c_data=encode_data(y_labels,x_data,output_dim)
y_c_labels_meningioma_tumor, x_c_data_meningioma_tumor=encode_data(y_labels_meningioma_tumor,x_data_meningioma_tumor,output_dim)
y_c_labels_glioma_tumor, x_c_data_glioma_tumor=encode_data(y_labels_glioma_tumor,x_data_glioma_tumor,output_dim)
y_c_labels_pituitary_tumor, x_c_data_pituitary_tumor=encode_data(y_labels_pituitary_tumor,x_data_pituitary_tumor,output_dim)

# 2.2. partition the data into training and testing sets
# partition the data into training and testing sets
# 90% of the data for training and the remaining 10% for testing
(trainX, testX, trainY, testY) = train_test_split(x_c_data, y_c_labels,
          test_size=0.1, stratify=y_c_labels, random_state=42)

(trainX_meningioma_tumor, testX_meningioma_tumor, trainY_meningioma_tumor, testY_meningioma_tumor) = train_test_split(x_c_data_meningioma_tumor, y_c_labels_meningioma_tumor,test_size=0.1, stratify=y_c_labels_meningioma_tumor, random_state=42)

(trainX_glioma_tumor, testX_glioma_tumor, trainY_glioma_tumor, testY_glioma_tumor) = train_test_split(x_c_data_glioma_tumor, y_c_labels_glioma_tumor,
          test_size=0.1, stratify=y_c_labels_glioma_tumor, random_state=42)

(trainX_pituitary_tumor, testX_pituitary_tumor, trainY_pituitary_tumor, testY_pituitary_tumor) = train_test_split(x_c_data_pituitary_tumor, y_c_labels_pituitary_tumor,
          test_size=0.1, stratify=y_c_labels_pituitary_tumor, random_state=42)

聯邦學習模型初始化

In [6]:
# build a training model
#model_meningioma_tumor
model = {'meningioma_tumor': Graph_pytorch()}
model['meningioma_tumor'].set_output_size(output_dim)
model['meningioma_tumor'].set_input_size([width_size,height_size,depth_size])
model['meningioma_tumor'].init_op_sampling()
model['meningioma_tumor'].get_linear_relation_layer_input_size()
model['meningioma_tumor'].init_op_linear()
model['meningioma_tumor'].init_optimizer()

In [7]:
# model_glioma_tumor
model['glioma_tumor'] = Graph_pytorch()
model['glioma_tumor'].set_output_size(output_dim)
model['glioma_tumor'].set_input_size([width_size,height_size,depth_size])
model['glioma_tumor'].init_op_sampling()
model['glioma_tumor'].get_linear_relation_layer_input_size()
model['glioma_tumor'].init_op_linear()
model['glioma_tumor'].init_optimizer()

In [8]:
# model_pituitary_tumor
model['pituitary_tumor'] = Graph_pytorch()
model['pituitary_tumor'].set_output_size(output_dim)
model['pituitary_tumor'].set_input_size([width_size,height_size,depth_size])
model['pituitary_tumor'].init_op_sampling()
model['pituitary_tumor'].get_linear_relation_layer_input_size()
model['pituitary_tumor'].init_op_linear()
model['pituitary_tumor'].init_optimizer()

訓練

In [9]:
def train(model, loss_func, train_loader, epochs):
    # training
    for i in range(epochs):
        model.train()
        y_train, x_train = train_loader
        # 轉換矩陣 from (batch, width, height, depth)
        #      to  (batch, depth, width, height)
        x_train = x_train.reshape(-1, width_size, height_size, depth_size)
        x_train = np.transpose(x_train, (0, 3, 1, 2))
        X_s = model.to_placeholder(x_train)
        Y_s = model.to_placeholder(y_train)
        # get y_pred
        y_prediction=model.forward(X_s)
        # get loss = y - y_pred
        loss = loss_func(Y_s, y_prediction)
        # adjust a,b
        model.backward(loss)

交換

In [10]:
def exchange(model):
    # Convolutional layer parameter sharing
    for index in model['meningioma_tumor'].op_sampling.keys():
        try:
            # init sum of parameters
            Sum = 0
            # FedAvg
            for i in model:
                Sum += model[i].op_sampling[index].weight.data
            Avg = Sum / len(model)
            # model copy parameters
            for i in model:
                model[i].op_sampling[index].weight.data.copy_(Avg)
        except:
            continue

    # Linear layer parameter sharing
    for index in model['meningioma_tumor'].op_linear.keys():
        try:
            # init sum of parameters
            Sum = 0
            # FedAvg
            for i in model:
                Sum += model[i].op_linear[index].weight.data
            Avg = Sum / len(model)
            # model copy parameters
            for i in model:
                model[i].op_linear[index].weight.data.copy_(Avg)
        except:
            continue

測試

In [11]:
def test(model, X, Y):
    accuracy = {}
    for i in model:
        model[i].eval()
        y_test_prediction = model[i].forward(X[i])
        y_truth = torch.argmax(Y[i],dim=1).cpu().detach().numpy()
        y_pred = torch.argmax(y_test_prediction,dim=1).cpu().detach().numpy()
        accuracy[i] = accuracy_score(y_truth,y_pred)
    return accuracy

In [12]:
# pychart
chart_epoch = []
chart_x_meningioma_tumor = []
chart_x_glioma_tumor = []
chart_x_pituitary_tumor = []

# 取樣
_sample_y, _sample_x = get_sample(testY, testX, 128)
# testing data transform
x_test = _sample_x.reshape(-1, width_size, height_size, depth_size)
x_test = np.transpose(x_test, (0, 3, 1, 2))
# model_meningioma_tumor
X_test = {'meningioma_tumor': model['meningioma_tumor'].to_placeholder(x_test)}
Y_test = {'meningioma_tumor': model['meningioma_tumor'].to_placeholder(_sample_y)}
# model_glioma_tumor
X_test['glioma_tumor'] = model['glioma_tumor'].to_placeholder(x_test)
Y_test['glioma_tumor'] = model['glioma_tumor'].to_placeholder(_sample_y)
# model_pituitary_tumor
X_test['pituitary_tumor'] = model['pituitary_tumor'].to_placeholder(x_test)
Y_test['pituitary_tumor'] = model['pituitary_tumor'].to_placeholder(_sample_y)

print("\n start federal learning....", flush=True)

#交換次數及每次交換中訓練次數設置
exchange_epochs = 300 # 交換資訊次數
epochs = 80 # 每輪交換資訊後機構各自訓練次數
loss_func = nn.SmoothL1Loss()  # 初始化損失函數
batch_size = 64

pbar = tqdm(range(exchange_epochs))

for exchange_epoch in pbar:
    # training
    train(model['meningioma_tumor'], loss_func, get_sample(trainY_meningioma_tumor, trainX_meningioma_tumor, batch_size), epochs)
    train(model['glioma_tumor'], loss_func, get_sample(trainY_glioma_tumor, trainX_glioma_tumor, batch_size), epochs)
    train(model['pituitary_tumor'], loss_func, get_sample(trainY_pituitary_tumor, trainX_pituitary_tumor, batch_size), epochs)

    # exchange
    exchange(model)

    # 3. testing
    # model_meningioma_tumor
    accuracy = test(model, X_test, Y_test)

    pbar.set_description(f'Processing {exchange_epoch}, accuracy_meningioma_tumor: {accuracy["meningioma_tumor"]}, accuracy_glioma_tumor: {accuracy["glioma_tumor"]}, accuracy_pituitary_tumor: {accuracy["pituitary_tumor"]}')

    # 3.3.4. store the statistic chart
    chart_epoch.append(str(exchange_epoch))
    chart_x_meningioma_tumor.append(accuracy["meningioma_tumor"])
    chart_x_glioma_tumor.append(accuracy["glioma_tumor"])
    chart_x_pituitary_tumor.append(accuracy["pituitary_tumor"])
pass

# save log
df = pd.DataFrame (chart_epoch,columns=['epoch'])
df['accuracy_meningioma_tumor']=chart_x_meningioma_tumor
df['accuracy_glioma_tumor']=chart_x_glioma_tumor
df['accuracy_pituitary_tumor']=chart_x_pituitary_tumor

df.to_csv (MY_PATH_TMP+'fedavgmodel_bs64_train80_exchange300_dropout1_fc2_test3_SmoothL1Loss_mean.csv', index = False, header=True)


 start federal learning....


Processing 299, accuracy_meningioma_tumor: 0.625, accuracy_glioma_tumor: 0.7109375, accuracy_pituitary_tumor: 0.6953125: 100%|██████████| 300/300 [1:13:50<00:00, 14.77s/it]    


將訓練好模型載入並存入model以方便之後重開可繼續測試

In [40]:
y_val, x_val = load_sample('data/val.p')
y_val, x_val = encode_data(y_val, x_val, output_dim)

_val_y, _val_x = get_sample(y_val, x_val, 128)
# testing data transform
_X_val = _val_x.reshape(-1, width_size, height_size, depth_size)
_X_val = np.transpose(_X_val, (0, 3, 1, 2))
# model_meningioma_tumor
X_val = {'meningioma_tumor': model['meningioma_tumor'].to_placeholder(_X_val)}
Y_val = {'meningioma_tumor': model['meningioma_tumor'].to_placeholder(_val_y)}
# model_meningioma_tumor
X_val['glioma_tumor'] = model['glioma_tumor'].to_placeholder(_X_val)
Y_val['glioma_tumor'] = model['glioma_tumor'].to_placeholder(_val_y)
# model_meningioma_tumor
X_val['pituitary_tumor'] = model['pituitary_tumor'].to_placeholder(_X_val)
Y_val['pituitary_tumor'] = model['pituitary_tumor'].to_placeholder(_val_y)

In [41]:
accuracy = test(model, X_val, Y_val)

print(f'accuracy_meningioma_tumor: {accuracy["meningioma_tumor"]}, accuracy_glioma_tumor: {accuracy["glioma_tumor"]}, accuracy_pituitary_tumor: {accuracy["pituitary_tumor"]}')

accuracy_meningioma_tumor: 0.671875, accuracy_glioma_tumor: 0.734375, accuracy_pituitary_tumor: 0.703125


In [42]:
# 6. save model
torch.save(model['meningioma_tumor'], MY_PATH_TMP+'FL_pytorch_model_meningioma_tumor.pkl')
torch.save(model['glioma_tumor'], MY_PATH_TMP+'FL_pytorch_model_glioma_tumor.pkl')
torch.save(model['pituitary_tumor'], MY_PATH_TMP+'FL_pytorch_model_pituitary_tumor.pkl')

In [43]:
FL_model_meningioma_tumor = torch.load(MY_PATH_TMP+'FL_pytorch_model_glioma_tumor.pkl')
FL_model_glioma_tumor = torch.load(MY_PATH_TMP+'FL_pytorch_model_glioma_tumor.pkl')
FL_model_pituitary_tumor = torch.load(MY_PATH_TMP+'FL_pytorch_model_pituitary_tumor.pkl')

測試

In [44]:
# 5. testing a picture
PicDir= "data/val/3/image(22).jpg"
#display(Image.open(PicDir))
NArray_2D  = Image_to_Tensor(PicDir,width_size,height_size)

In [45]:
def pred(prediction):
    if prediction==0:
        return 'normal'
    elif prediction==1:
        return 'meningioma_tumor'
    elif prediction==2:
        return 'glioma_tumor'
    elif prediction==3:
        return 'pituitary_tumor'

In [47]:
#FL_model_meningioma_tumor
prediction=FL_model_meningioma_tumor.forward(NArray_2D)
print(prediction.clone().detach().numpy())
print(f"Predictions for models with only meningioma tumor:{pred(np.argmax(prediction.clone().detach().numpy()))}")

#FL_model_glioma_tumor
prediction=FL_model_glioma_tumor.forward(NArray_2D)
print(prediction.clone().detach().numpy())
print(f"Predictions for models with only glioma tumor:{pred(np.argmax(prediction.clone().detach().numpy()))}")

#FL_model_pituitary_tumor
prediction=FL_model_pituitary_tumor.forward(NArray_2D)
print(prediction.clone().detach().numpy())
print(f"Predictions for models with only pituitary tumor:{pred(np.argmax(prediction.clone().detach().numpy()))}")

[[0.50255835 0.00369454 0.48810476 0.00564238]]
Predictions for models with only meningioma tumor:normal
[[2.5568905e-10 3.3274659e-12 2.2846429e-23 1.0000000e+00]]
Predictions for models with only glioma tumor:pituitary_tumor
[[3.4979992e-12 3.2979428e-14 1.3247615e-27 1.0000000e+00]]
Predictions for models with only pituitary tumor:pituitary_tumor
